### Environment

In [ ]:
# 切换conda虚拟环境
# source /opt/conda/bin/activate

### Dataset

In [ ]:
# 数据集
!unzip /home/share/MOT17Det.zip -d /home/ByteTrack/datasets/mot
!unzip /home/share/MOT20.zip -d /home/ByteTrack/datasets

In [ ]:
%cd /home/ByteTrack
!python tools/convert_mot17_to_coco.py
!python tools/convert_mot20_to_coco.py
!python tools/convert_crowdhuman_to_coco.py
!python tools/convert_cityperson_to_coco.py
!python tools/convert_ethz_to_coco.py

In [ ]:
!python tools/mix_data_ablation.py
!python tools/mix_data_test_mot17.py
!python tools/mix_data_test_mot20.py

### Train

In [ ]:
# Train MOT17 test model (MOT17 train, CrowdHuman, Cityperson and ETHZ)
!python tools/train.py -f exps/example/mot/yolox_x_mix_det.py -d 8 -b 48 --fp16 -o -c pretrained/yolox_x.pth.tar

In [ ]:
# Train MOT20 test model (MOT20 train, CrowdHuman)
!python tools/train.py -f exps/example/mot/yolox_x_mix_mot20_ch.py -d 1 -b 4 --fp16 -o -c pretrained/yolox_x.pth.tar
# !python tools/train.py -f exps/example/mot/yolox_x_mix_mot20_ch.py -d 1 -b 4 --resume --fp16 -o -c YOLOX_outputs/yolox_x_mix_mot20_ch/latest_ckpt.pth.tar

### Inference

In [ ]:
# Evaluation on MOT17 half val
!python tools/track.py -f exps/example/mot/yolox_x_ablation.py -c pretrained/bytetrack_ablation.pth.tar -b 1 -d 1 --fp16 --fuse

In [3]:
%cd /home/ByteTrack
!python tools/demo_track.py image -f exps/example/mot/yolox_x_mix_det.py -c YOLOX_outputs/yolox_x_mix_mot20_ch/latest_ckpt.pth.tar --fp16 --fuse --save_result

/home/ByteTrack
2022-12-02 02:41:08.312 | INFO     | __main__:main:316 - Args: Namespace(aspect_ratio_thresh=1.6, camid=0, ckpt='YOLOX_outputs/yolox_x_mix_mot20_ch/latest_ckpt.pth.tar', conf=None, demo='image', device=device(type='cuda'), exp_file='exps/example/mot/yolox_x_mix_det.py', experiment_name='yolox_x_mix_det', fp16=True, fps=30, fuse=True, match_thresh=0.8, min_box_area=10, mot20=False, name=None, nms=None, path='./datasets/test', save_result=True, track_buffer=30, track_thresh=0.5, trt=False, tsize=None)
2022-12-02 02:41:11.514 | INFO     | __main__:main:326 - Model Summary: Params: 99.00M, Gflops: 793.21
2022-12-02 02:41:11.516 | INFO     | __main__:main:334 - loading checkpoint
2022-12-02 02:41:11.841 | INFO     | __main__:main:338 - loaded checkpoint done.
2022-12-02 02:41:11.841 | INFO     | __main__:main:341 - 	Fusing model...
/opt/conda/lib/python3.7/site-packages/torch/nn/modules/module.py:390: UserWarning: The .grad attribute of a Tensor that is not a leaf Tensor is 

### Test

In [ ]:
# test on MOT20
%cd /home/ByteTrack
!python tools/track.py -f exps/example/mot/yolox_x_mix_mot20_ch.py -c YOLOX_outputs/yolox_x_mix_mot20_ch/latest_ckpt.pth.tar -b 1 -d 1 --fp16 --fuse --match_thresh 0.7 --mot20
!python tools/interpolation.py